In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from dotenv import load_dotenv
import glob
from mymodule import myfunc as mf
load_dotenv()
imgout = os.environ["imgout"]
root_dir = os.environ["root_dir"]

In [16]:
import lic
from PIL import Image

# ... get x and y arrays from somewhere ...
for i in range(1):
    # for x in glob.glob(root_dir + f"imgout/snap49split1/magfieldx/{i}/*.npy"):
    for x in glob.glob(root_dir + f"snap/snap49/magfieldx/09/*")[-1:]:
        xfile = mf.load(x)
        yfile = mf.load(x.replace("magfieldx", "magfieldy"))
        # xfile = mf.im_interpolate(xfile, 108, 432)
        # yfile = mf.im_interpolate(yfile, 108, 432)
        out = root_dir + f"imgout/lictest/{i}/{i}_ohnostrm{x[-5:]}.png"##snap
        # out = root_dir + f"imgout/lictest/{i}/{i}_ohnostrm{x[-10:-4]}.png"###split
        im = lic.lic(xfile, yfile, length=15)#, contrast=True)
        im = mf.lic2image(im)
        # im.show()
        im.save(out)


generate lics: 100%|██████████| 7/7 [00:01<00:00,  4.34it/s]
C:\Users\lkxl1\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.


1.0